Copyright 2022 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Android Topics API Classifier Execution Demo

This colab is used to show a simple demo how to load Topics API's tflite model and compute the app topics.

Before running the notebook:

1. You will need to download the tflite model file from your browser. Click [here](https://dl.google.com/mdi-serving/topics-classifier/2/model.tflite) to download the model automatically.

2. Once you have the model, you will need to upload it locally to the colab. (See the folder icon on the left to upload files. Please make sure the model name is "model.tflite")

Note: Topics API's taxonomy list can be found in this [link](https://github.com/privacysandbox/topics-android/blob/main/taxonomy_v2.md)

In [ ]:
#@title Install tflite dependency and import related dependencies {display-mode: "form"}
%%capture
!pip install -q tflite-support

import re
import math
from tflite_support.task import text
from tflite_support.task import core

_NUMBER_OF_TOP_TOPICS = 3
_THRESHOLD = 0.1

In [ ]:
#@title Click "Show code" to view the Android Topics API taxonomy list {display-mode: "form"}

cat_map = {
    10001: '/Arts & Entertainment',
    10002: '/Arts & Entertainment/Acting & Theater',
    10003: '/Arts & Entertainment/Anime & Manga',
    10004: '/Arts & Entertainment/Cartoons',
    10005: '/Arts & Entertainment/Comics',
    10006: '/Arts & Entertainment/Concerts & Music Festivals',
    10007: '/Arts & Entertainment/Dance',
    10008: '/Arts & Entertainment/Entertainment Industry',
    10009: '/Arts & Entertainment/Fun & Trivia',
    10010: '/Arts & Entertainment/Fun & Trivia/Fun Tests & Silly Surveys',
    10011: '/Arts & Entertainment/Humor',
    10012: '/Arts & Entertainment/Humor/Funny Pictures & Videos',
    10013: '/Arts & Entertainment/Humor/Live Comedy',
    10014: '/Arts & Entertainment/Live Sporting Events',
    10015: '/Arts & Entertainment/Magic',
    10016: '/Arts & Entertainment/Movie Listings & Theater Showtimes',
    10017: '/Arts & Entertainment/Movies',
    10018: '/Arts & Entertainment/Movies/Action & Adventure Films',
    10019: '/Arts & Entertainment/Movies/Animated Films',
    10020: '/Arts & Entertainment/Movies/Comedy Films',
    10021: '/Arts & Entertainment/Movies/Cult & Indie Films',
    10022: '/Arts & Entertainment/Movies/Documentary Films',
    10023: '/Arts & Entertainment/Movies/Drama Films',
    10024: '/Arts & Entertainment/Movies/Family Films',
    10025: '/Arts & Entertainment/Movies/Horror Films',
    10026: '/Arts & Entertainment/Movies/Romance Films',
    10027: '/Arts & Entertainment/Movies/Thriller, Crime & Mystery Films',
    10028: '/Arts & Entertainment/Music & Audio',
    10029: '/Arts & Entertainment/Music & Audio/Blues',
    10030: '/Arts & Entertainment/Music & Audio/Classical Music',
    10031: '/Arts & Entertainment/Music & Audio/Country Music',
    10032: '/Arts & Entertainment/Music & Audio/Dance & Electronic Music',
    10033: '/Arts & Entertainment/Music & Audio/Folk & Traditional Music',
    10034: '/Arts & Entertainment/Music & Audio/Jazz',
    10035: '/Arts & Entertainment/Music & Audio/Music Streams & Downloads',
    10036: '/Arts & Entertainment/Music & Audio/Music Videos',
    10037: '/Arts & Entertainment/Music & Audio/Musical Instruments',
    10038: '/Arts & Entertainment/Music & Audio/Musical Instruments/Pianos & Keyboards',
    10039: '/Arts & Entertainment/Music & Audio/Pop Music',
    10040: '/Arts & Entertainment/Music & Audio/Radio',
    10041: '/Arts & Entertainment/Music & Audio/Radio/Talk Radio',
    10042: '/Arts & Entertainment/Music & Audio/Rap & Hip-Hop',
    10043: '/Arts & Entertainment/Music & Audio/Rock Music',
    10044: '/Arts & Entertainment/Music & Audio/Rock Music/Classic Rock & Oldies',
    10045: '/Arts & Entertainment/Music & Audio/Rock Music/Hard Rock & Progressive',
    10046: '/Arts & Entertainment/Music & Audio/Rock Music/Indie & Alternative Music',
    10047: '/Arts & Entertainment/Music & Audio/Samples & Sound Libraries',
    10048: '/Arts & Entertainment/Music & Audio/Soul & R&B',
    10049: '/Arts & Entertainment/Music & Audio/Soundtracks',
    10050: '/Arts & Entertainment/Music & Audio/World Music',
    10051: '/Arts & Entertainment/Music & Audio/World Music/Reggae & Caribbean Music',
    10052: '/Arts & Entertainment/Online Image Galleries',
    10053: '/Arts & Entertainment/Online Video',
    10054: '/Arts & Entertainment/Online Video/Live Video Streaming',
    10055: '/Arts & Entertainment/Online Video/Movie & TV Streaming',
    10056: '/Arts & Entertainment/Opera',
    10057: '/Arts & Entertainment/TV Guides & Reference',
    10058: '/Arts & Entertainment/TV Networks & Stations',
    10059: '/Arts & Entertainment/TV Shows & Programs',
    10060: '/Arts & Entertainment/TV Shows & Programs/TV Comedies',
    10061: '/Arts & Entertainment/TV Shows & Programs/TV Documentary & Nonfiction',
    10062: '/Arts & Entertainment/TV Shows & Programs/TV Dramas',
    10063: '/Arts & Entertainment/TV Shows & Programs/TV Dramas/TV Soap Operas',
    10064: '/Arts & Entertainment/TV Shows & Programs/TV Family-Oriented Shows',
    10065: '/Arts & Entertainment/TV Shows & Programs/TV Reality Shows',
    10066: '/Arts & Entertainment/TV Shows & Programs/TV Sci-Fi & Fantasy Shows',
    10067: '/Arts & Entertainment/Visual Art & Design',
    10068: '/Arts & Entertainment/Visual Art & Design/Design',
    10069: '/Arts & Entertainment/Visual Art & Design/Painting',
    10070: '/Arts & Entertainment/Visual Art & Design/Photographic & Digital Arts',
    10071: '/Autos & Vehicles',
    10072: '/Autos & Vehicles/Cargo Trucks & Trailers',
    10073: '/Autos & Vehicles/Classic Vehicles',
    10074: '/Autos & Vehicles/Custom & Performance Vehicles',
    10075: '/Autos & Vehicles/Gas Prices & Vehicle Fueling',
    10076: '/Autos & Vehicles/Motor Vehicles (By Type)',
    10077: '/Autos & Vehicles/Motor Vehicles (By Type)/Autonomous Vehicles',
    10078: '/Autos & Vehicles/Motor Vehicles (By Type)/Convertibles',
    10079: '/Autos & Vehicles/Motor Vehicles (By Type)/Coupes',
    10080: '/Autos & Vehicles/Motor Vehicles (By Type)/Hatchbacks',
    10081: '/Autos & Vehicles/Motor Vehicles (By Type)/Hybrid & Alternative Vehicles',
    10082: '/Autos & Vehicles/Motor Vehicles (By Type)/Luxury Vehicles',
    10083: '/Autos & Vehicles/Motor Vehicles (By Type)/Microcars & Subcompacts',
    10084: '/Autos & Vehicles/Motor Vehicles (By Type)/Motorcycles',
    10085: '/Autos & Vehicles/Motor Vehicles (By Type)/Off-Road Vehicles',
    10086: '/Autos & Vehicles/Motor Vehicles (By Type)/Pickup Trucks',
    10087: '/Autos & Vehicles/Motor Vehicles (By Type)/Scooters & Mopeds',
    10088: '/Autos & Vehicles/Motor Vehicles (By Type)/Sedans',
    10089: '/Autos & Vehicles/Motor Vehicles (By Type)/Station Wagons',
    10090: '/Autos & Vehicles/Motor Vehicles (By Type)/SUVs & Crossovers',
    10091: '/Autos & Vehicles/Motor Vehicles (By Type)/SUVs & Crossovers/Crossovers',
    10092: '/Autos & Vehicles/Motor Vehicles (By Type)/Vans & Minivans',
    10093: '/Autos & Vehicles/Towing & Roadside Assistance',
    10094: '/Autos & Vehicles/Vehicle & Traffic Safety',
    10095: '/Autos & Vehicles/Vehicle Parts & Accessories',
    10096: '/Autos & Vehicles/Vehicle Repair & Maintenance',
    10097: '/Autos & Vehicles/Vehicle Shopping',
    10098: '/Autos & Vehicles/Vehicle Shopping/Used Vehicles',
    10099: '/Autos & Vehicles/Vehicle Shows',
    10100: '/Beauty & Fitness',
    10101: '/Beauty & Fitness/Body Art',
    10102: '/Beauty & Fitness/Face & Body Care',
    10103: '/Beauty & Fitness/Face & Body Care/Antiperspirants, Deodorants & Body Sprays',
    10104: '/Beauty & Fitness/Face & Body Care/Bath & Body Products',
    10105: '/Beauty & Fitness/Face & Body Care/Clean Beauty',
    10106: '/Beauty & Fitness/Face & Body Care/Make-Up & Cosmetics',
    10107: '/Beauty & Fitness/Face & Body Care/Nail Care Products',
    10108: '/Beauty & Fitness/Face & Body Care/Perfumes & Fragrances',
    10109: '/Beauty & Fitness/Face & Body Care/Razors & Shavers',
    10110: '/Beauty & Fitness/Fashion & Style',
    10111: '/Beauty & Fitness/Fitness',
    10112: '/Beauty & Fitness/Fitness/Bodybuilding',
    10113: '/Beauty & Fitness/Fitness/Fitness Instruction & Personal Training',
    10114: '/Beauty & Fitness/Fitness/Fitness Technology Products',
    10115: '/Beauty & Fitness/Hair Care',
    10116: '/Books & Literature',
    10117: "/Books & Literature/Children's Literature",
    10118: '/Books & Literature/E-Books',
    10119: '/Books & Literature/Magazines',
    10120: '/Books & Literature/Poetry',
    10121: '/Business & Industrial',
    10122: '/Business & Industrial/Advertising & Marketing',
    10123: '/Business & Industrial/Advertising & Marketing/Sales',
    10124: '/Business & Industrial/Agriculture & Forestry',
    10125: '/Business & Industrial/Agriculture & Forestry/Food Production',
    10126: '/Business & Industrial/Automotive Industry',
    10127: '/Business & Industrial/Aviation Industry',
    10128: '/Business & Industrial/Business Operations',
    10129: '/Business & Industrial/Business Operations/Flexible Work Arrangements',
    10130: '/Business & Industrial/Business Operations/Human Resources',
    10131: '/Business & Industrial/Commercial Lending',
    10132: '/Business & Industrial/Construction & Maintenance',
    10133: '/Business & Industrial/Construction & Maintenance/Civil Engineering',
    10134: '/Business & Industrial/Defense Industry',
    10135: '/Business & Industrial/Energy & Utilities',
    10136: '/Business & Industrial/Energy & Utilities/Water Supply & Treatment',
    10137: '/Business & Industrial/Hospitality Industry',
    10138: '/Business & Industrial/Manufacturing',
    10139: '/Business & Industrial/Metals & Mining',
    10140: '/Business & Industrial/MLM & Business Opportunities',
    10141: '/Business & Industrial/Pharmaceuticals & Biotech',
    10142: '/Business & Industrial/Printing & Publishing',
    10143: '/Business & Industrial/Retail Trade',
    10144: '/Business & Industrial/Venture Capital',
    10145: '/Computers & Electronics',
    10146: '/Computers & Electronics/Computer Peripherals',
    10147: '/Computers & Electronics/Computer Peripherals/Printers',
    10148: '/Computers & Electronics/Computer Security',
    10149: '/Computers & Electronics/Computer Security/Antivirus & Malware',
    10150: '/Computers & Electronics/Computer Security/Network Security',
    10151: '/Computers & Electronics/Consumer Electronics',
    10152: '/Computers & Electronics/Consumer Electronics/Cameras & Camcorders',
    10153: '/Computers & Electronics/Consumer Electronics/GPS & Navigation',
    10154: '/Computers & Electronics/Consumer Electronics/Home Automation',
    10155: '/Computers & Electronics/Consumer Electronics/Home Theater Systems',
    10156: '/Computers & Electronics/Consumer Electronics/MP3 & Portable Media Players',
    10157: '/Computers & Electronics/Consumer Electronics/Wearable Technology',
    10158: '/Computers & Electronics/Data Backup & Recovery',
    10159: '/Computers & Electronics/Desktop Computers',
    10160: '/Computers & Electronics/Laptops & Notebooks',
    10161: '/Computers & Electronics/Networking',
    10162: '/Computers & Electronics/Networking/Distributed & Cloud Computing',
    10163: '/Computers & Electronics/Programming',
    10164: '/Computers & Electronics/Software',
    10165: '/Computers & Electronics/Software/Audio & Music Software',
    10166: '/Computers & Electronics/Software/Business & Productivity Software',
    10167: '/Computers & Electronics/Software/Business & Productivity Software/Calendar & Scheduling Software',
    10168: '/Computers & Electronics/Software/Business & Productivity Software/Collaboration & Conferencing Software',
    10169: '/Computers & Electronics/Software/Business & Productivity Software/Presentation Software',
    10170: '/Computers & Electronics/Software/Business & Productivity Software/Spreadsheet Software',
    10171: '/Computers & Electronics/Software/Business & Productivity Software/Word Processing Software',
    10172: '/Computers & Electronics/Software/Desktop Publishing',
    10173: '/Computers & Electronics/Software/Desktop Publishing/Fonts',
    10174: '/Computers & Electronics/Software/Download Managers',
    10175: '/Computers & Electronics/Software/Freeware & Shareware',
    10176: '/Computers & Electronics/Software/Graphics & Animation Software',
    10177: '/Computers & Electronics/Software/Intelligent Personal Assistants',
    10178: '/Computers & Electronics/Software/Media Players',
    10179: '/Computers & Electronics/Software/Monitoring Software',
    10180: '/Computers & Electronics/Software/Operating Systems',
    10181: '/Computers & Electronics/Software/Photo & Video Software',
    10182: '/Computers & Electronics/Software/Photo & Video Software/Photo Software',
    10183: '/Computers & Electronics/Software/Photo & Video Software/Video Software',
    10184: '/Computers & Electronics/Software/Software Utilities',
    10185: '/Computers & Electronics/Software/Web Browsers',
    10186: '/Finance',
    10187: '/Finance/Accounting & Auditing',
    10188: '/Finance/Accounting & Auditing/Tax Preparation & Planning',
    10189: '/Finance/Banking',
    10190: '/Finance/Banking/Money Transfer & Wire Services',
    10191: '/Finance/Credit & Lending',
    10192: '/Finance/Credit & Lending/Credit Cards',
    10193: '/Finance/Credit & Lending/Home Financing',
    10194: '/Finance/Credit & Lending/Personal Loans',
    10195: '/Finance/Credit & Lending/Student Loans & College Financing',
    10196: '/Finance/Financial Planning & Management',
    10197: '/Finance/Financial Planning & Management/Retirement & Pension',
    10198: '/Finance/Grants, Scholarships & Financial Aid',
    10199: '/Finance/Grants, Scholarships & Financial Aid/Study Grants & Scholarships',
    10200: '/Finance/Insurance',
    10201: '/Finance/Insurance/Auto Insurance',
    10202: '/Finance/Insurance/Health Insurance',
    10203: '/Finance/Insurance/Home Insurance',
    10204: '/Finance/Insurance/Life Insurance',
    10205: '/Finance/Insurance/Travel Insurance',
    10206: '/Finance/Investing',
    10207: '/Finance/Investing/Commodities & Futures Trading',
    10208: '/Finance/Investing/Currencies & Foreign Exchange',
    10209: '/Finance/Investing/Hedge Funds',
    10210: '/Finance/Investing/Mutual Funds',
    10211: '/Finance/Investing/Stocks & Bonds',
    10212: '/Food & Drink',
    10213: '/Food & Drink/Cooking & Recipes',
    10214: '/Food & Drink/Cooking & Recipes/BBQ & Grilling',
    10215: '/Food & Drink/Cooking & Recipes/Cuisines',
    10216: '/Food & Drink/Cooking & Recipes/Cuisines/Vegetarian Cuisine',
    10217: '/Food & Drink/Cooking & Recipes/Cuisines/Vegetarian Cuisine/Vegan Cuisine',
    10218: '/Food & Drink/Cooking & Recipes/Healthy Eating',
    10219: '/Food & Drink/Food & Grocery Retailers',
    10220: '/Games',
    10221: '/Games/Arcade & Coin-Op Games',
    10222: '/Games/Billiards',
    10223: '/Games/Board Games',
    10224: '/Games/Board Games/Chess & Abstract Strategy Games',
    10225: '/Games/Card Games',
    10226: '/Games/Card Games/Collectible Card Games',
    10227: '/Games/Computer & Video Games',
    10228: '/Games/Computer & Video Games/Action & Platform Games',
    10229: '/Games/Computer & Video Games/Adventure Games',
    10230: '/Games/Computer & Video Games/Casual Games',
    10231: '/Games/Computer & Video Games/Competitive Video Gaming',
    10232: '/Games/Computer & Video Games/Driving & Racing Games',
    10233: '/Games/Computer & Video Games/Fighting Games',
    10234: '/Games/Computer & Video Games/Gaming Reference & Reviews',
    10235: '/Games/Computer & Video Games/Gaming Reference & Reviews/Video Game Cheats & Hints',
    10236: '/Games/Computer & Video Games/Massively Multiplayer Games',
    10237: '/Games/Computer & Video Games/Music & Dance Games',
    10238: '/Games/Computer & Video Games/Sandbox Games',
    10239: '/Games/Computer & Video Games/Shooter Games',
    10240: '/Games/Computer & Video Games/Simulation Games',
    10241: '/Games/Computer & Video Games/Simulation Games/Business & Tycoon Games',
    10242: '/Games/Computer & Video Games/Simulation Games/City Building Games',
    10243: '/Games/Computer & Video Games/Simulation Games/Life Simulation Games',
    10244: '/Games/Computer & Video Games/Simulation Games/Vehicle Simulators',
    10245: '/Games/Computer & Video Games/Sports Games',
    10246: '/Games/Computer & Video Games/Sports Games/Sports Management Games',
    10247: '/Games/Computer & Video Games/Strategy Games',
    10248: '/Games/Computer & Video Games/Video Game Mods & Add-Ons',
    10249: '/Games/Educational Games',
    10250: '/Games/Family-Oriented Games & Activities',
    10251: '/Games/Family-Oriented Games & Activities/Drawing & Coloring',
    10252: '/Games/Family-Oriented Games & Activities/Dress-Up & Fashion Games',
    10253: '/Games/Puzzles & Brainteasers',
    10254: '/Games/Roleplaying Games',
    10255: '/Games/Table Tennis',
    10256: '/Games/Tile Games',
    10257: '/Games/Word Games',
    10258: '/Hobbies & Leisure',
    10259: '/Hobbies & Leisure/Anniversaries',
    10260: '/Hobbies & Leisure/Birthdays & Name Days',
    10261: '/Hobbies & Leisure/Diving & Underwater Activities',
    10262: '/Hobbies & Leisure/Fiber & Textile Arts',
    10263: '/Hobbies & Leisure/Outdoors',
    10264: '/Hobbies & Leisure/Outdoors/Fishing',
    10265: '/Hobbies & Leisure/Outdoors/Hunting & Shooting',
    10266: '/Hobbies & Leisure/Paintball',
    10267: '/Hobbies & Leisure/Radio Control & Modeling',
    10268: '/Hobbies & Leisure/Weddings',
    10269: '/Home & Garden',
    10270: '/Home & Garden/Gardening',
    10271: '/Home & Garden/Home & Interior Decor',
    10272: '/Home & Garden/Home Appliances',
    10273: '/Home & Garden/Home Improvement',
    10274: '/Home & Garden/Home Safety & Security',
    10275: '/Home & Garden/Household Supplies',
    10276: '/Home & Garden/Landscape Design',
    10277: '/Internet & Telecom',
    10278: '/Internet & Telecom/Email & Messaging',
    10279: '/Internet & Telecom/Email & Messaging/Email',
    10280: '/Internet & Telecom/Email & Messaging/Text & Instant Messaging',
    10281: '/Internet & Telecom/Email & Messaging/Voice & Video Chat',
    10282: '/Internet & Telecom/ISPs',
    10283: '/Internet & Telecom/Phone Service Providers',
    10284: '/Internet & Telecom/Ringtones & Mobile Themes',
    10285: '/Internet & Telecom/Search Engines',
    10286: '/Internet & Telecom/Smart Phones',
    10287: '/Internet & Telecom/Teleconferencing',
    10288: '/Internet & Telecom/Web Apps & Online Tools',
    10289: '/Internet & Telecom/Web Services',
    10290: '/Internet & Telecom/Web Services/Cloud Storage',
    10291: '/Internet & Telecom/Web Services/Web Design & Development',
    10292: '/Internet & Telecom/Web Services/Web Hosting',
    10293: '/Jobs & Education',
    10294: '/Jobs & Education/Education',
    10295: '/Jobs & Education/Education/Academic Conferences & Publications',
    10296: '/Jobs & Education/Education/Colleges & Universities',
    10297: '/Jobs & Education/Education/Distance Learning',
    10298: '/Jobs & Education/Education/Early Childhood Education',
    10299: '/Jobs & Education/Education/Early Childhood Education/Preschool',
    10300: '/Jobs & Education/Education/Homeschooling',
    10301: '/Jobs & Education/Education/Standardized & Admissions Tests',
    10302: '/Jobs & Education/Education/Teaching & Classroom Resources',
    10303: '/Jobs & Education/Education/Vocational & Continuing Education',
    10304: '/Jobs & Education/Jobs',
    10305: '/Jobs & Education/Jobs/Career Resources & Planning',
    10306: '/Jobs & Education/Jobs/Job Listings',
    10307: '/Law & Government',
    10308: '/Law & Government/Crime & Justice',
    10309: '/Law & Government/Legal',
    10310: '/Law & Government/Legal/Legal Services',
    10311: '/News',
    10312: '/News/Economy News',
    10313: '/News/Local News',
    10314: '/News/Mergers & Acquisitions',
    10315: '/News/Newspapers',
    10316: '/News/Politics',
    10317: '/News/Sports News',
    10318: '/News/Weather',
    10319: '/News/World News',
    10320: '/Online Communities',
    10321: '/Online Communities/Clip Art & Animated GIFs',
    10322: '/Online Communities/Dating & Personals',
    10323: '/Online Communities/Feed Aggregation & Social Bookmarking',
    10324: '/Online Communities/File Sharing & Hosting',
    10325: '/Online Communities/Forum & Chat Providers',
    10326: '/Online Communities/Microblogging',
    10327: '/Online Communities/Photo & Video Sharing',
    10328: '/Online Communities/Photo & Video Sharing/Photo & Image Sharing',
    10329: '/Online Communities/Photo & Video Sharing/Video Sharing',
    10330: '/Online Communities/Skins, Themes & Wallpapers',
    10331: '/Online Communities/Social Network Apps & Add-Ons',
    10332: '/Online Communities/Social Networks',
    10333: '/People & Society',
    10334: '/People & Society/Family & Relationships',
    10335: '/People & Society/Family & Relationships/Ancestry & Genealogy',
    10336: '/People & Society/Family & Relationships/Marriage',
    10337: '/People & Society/Family & Relationships/Parenting',
    10338: '/People & Society/Family & Relationships/Parenting/Adoption',
    10339: '/People & Society/Family & Relationships/Parenting/Babies & Toddlers',
    10340: '/People & Society/Family & Relationships/Parenting/Child Internet Safety',
    10341: '/People & Society/Family & Relationships/Romance',
    10342: '/People & Society/Science Fiction & Fantasy',
    10343: '/Pets & Animals',
    10344: '/Pets & Animals/Pet Food & Pet Care Supplies',
    10345: '/Pets & Animals/Pets',
    10346: '/Pets & Animals/Pets/Birds',
    10347: '/Pets & Animals/Pets/Cats',
    10348: '/Pets & Animals/Pets/Dogs',
    10349: '/Pets & Animals/Pets/Fish & Aquaria',
    10350: '/Pets & Animals/Pets/Reptiles & Amphibians',
    10351: '/Pets & Animals/Veterinarians',
    10352: '/Real Estate',
    10353: '/Real Estate/Lots & Land',
    10354: '/Real Estate/Timeshares & Vacation Properties',
    10355: '/Reference',
    10356: '/Reference/Business & Personal Listings',
    10357: '/Reference/General Reference',
    10358: '/Reference/General Reference/Calculators & Reference Tools',
    10359: '/Reference/General Reference/Dictionaries & Encyclopedias',
    10360: '/Reference/General Reference/Educational Resources',
    10361: '/Reference/General Reference/How-To, DIY & Expert Content',
    10362: '/Reference/General Reference/Time & Calendars',
    10363: '/Reference/Language Resources',
    10364: '/Reference/Language Resources/Foreign Language Study',
    10365: '/Reference/Language Resources/Translation Tools & Resources',
    10366: '/Reference/Maps',
    10367: '/Science',
    10368: '/Science/Augmented & Virtual Reality',
    10369: '/Science/Biological Sciences',
    10370: '/Science/Biological Sciences/Genetics',
    10371: '/Science/Chemistry',
    10372: '/Science/Ecology & Environment',
    10373: '/Science/Geology',
    10374: '/Science/Machine Learning & Artificial Intelligence',
    10375: '/Science/Mathematics',
    10376: '/Science/Physics',
    10377: '/Science/Robotics',
    10378: '/Shopping',
    10379: '/Shopping/Antiques & Collectibles',
    10380: '/Shopping/Apparel',
    10381: "/Shopping/Apparel/Children's Clothing",
    10382: '/Shopping/Apparel/Costumes',
    10383: "/Shopping/Apparel/Men's Clothing",
    10384: "/Shopping/Apparel/Women's Clothing",
    10385: '/Shopping/Classifieds',
    10386: '/Shopping/Consumer Resources',
    10387: '/Shopping/Consumer Resources/Coupons & Discount Offers',
    10388: '/Shopping/Consumer Resources/Loyalty Cards & Programs',
    10389: '/Shopping/Consumer Resources/Technical Support & Repair',
    10390: '/Shopping/Flowers',
    10391: '/Shopping/Greeting Cards',
    10392: '/Shopping/Party & Holiday Supplies',
    10393: '/Shopping/Shopping Portals',
    10394: '/Sports',
    10395: '/Sports/American Football',
    10396: '/Sports/Australian Football',
    10397: '/Sports/Auto Racing',
    10398: '/Sports/Baseball',
    10399: '/Sports/Basketball',
    10400: '/Sports/Bowling',
    10401: '/Sports/Boxing',
    10402: '/Sports/Cheerleading',
    10403: '/Sports/College Sports',
    10404: '/Sports/Cricket',
    10405: '/Sports/Cycling',
    10406: '/Sports/Equestrian',
    10407: '/Sports/Extreme Sports',
    10408: '/Sports/Extreme Sports/Climbing & Mountaineering',
    10409: '/Sports/Fantasy Sports',
    10410: '/Sports/Golf',
    10411: '/Sports/Gymnastics',
    10412: '/Sports/Hockey',
    10413: '/Sports/Ice Skating',
    10414: '/Sports/Martial Arts',
    10415: '/Sports/Motorcycle Racing',
    10416: '/Sports/Olympics',
    10417: '/Sports/Rugby',
    10418: '/Sports/Running & Walking',
    10419: '/Sports/Skiing & Snowboarding',
    10420: '/Sports/Soccer',
    10421: '/Sports/Surfing',
    10422: '/Sports/Swimming',
    10423: '/Sports/Tennis',
    10424: '/Sports/Track & Field',
    10425: '/Sports/Volleyball',
    10426: '/Sports/Wrestling',
    10427: '/Travel & Transportation',
    10428: '/Travel & Transportation/Adventure Travel',
    10429: '/Travel & Transportation/Air Travel',
    10430: '/Travel & Transportation/Business Travel',
    10431: '/Travel & Transportation/Car Rentals',
    10432: '/Travel & Transportation/Cruises & Charters',
    10433: '/Travel & Transportation/Family Travel',
    10434: '/Travel & Transportation/Honeymoons & Romantic Getaways',
    10435: '/Travel & Transportation/Hotels & Accommodations',
    10436: '/Travel & Transportation/Long Distance Bus & Rail',
    10437: '/Travel & Transportation/Low Cost & Last Minute Travel',
    10438: '/Travel & Transportation/Luggage & Travel Accessories',
    10439: '/Travel & Transportation/Tourist Destinations',
    10440: '/Travel & Transportation/Tourist Destinations/Beaches & Islands',
    10441: '/Travel & Transportation/Tourist Destinations/Regional Parks & Gardens',
    10442: '/Travel & Transportation/Tourist Destinations/Theme Parks',
    10443: '/Travel & Transportation/Tourist Destinations/Zoos, Aquariums & Preserves',
    10444: '/Travel & Transportation/Traffic & Route Planners',
    10445: '/Travel & Transportation/Travel Agencies & Services',
    10446: '/Travel & Transportation/Travel Guides & Travelogues',
  }

In [ ]:
#@title Get the prediction of one description {display-mode: "form"}

def clean_description(description: str) -> str:
  """Cleans the app description.

  Converts the string to lower case, removes urls from description, removes
  @mentions annotation, removes html tags.

  Args:
    description: The string to describe the app information.

  Returns:
    Cleaned app description.
  """
  # Converts to lower case.
  description = description.lower()
  # Removes urls.
  description = re.sub(
      r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
      "", description, flags=re.MULTILINE)
  description = re.sub(
      r"www.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
      "", description, flags=re.MULTILINE)
  # Removes @mentions.
  description = re.sub(r"@[A-Za-z0-9]+", "", description)

  # Removes html tags.
  description = re.sub("\\<.*?\\>", "", description)

  # Removes new line and tab.
  description = description.replace("\n", " ").replace("\t", " ")

  # Removes multiple space.
  description = re.sub(" +", " ", description)

  return description

description = "Google Drive, part of Google Workspace, is a safe place to back up and access all your files from any device. Easily invite others to view, edit, or leave comments on any of your files or folders.  With Drive, you can:  \u2022 Safely store and access your files anywhere \u2022 Quickly access recent and important files \u2022 Search for files by name and content \u2022 Share and set permissions for files and folders \u2022 View your content on the go while offline \u2022 Receive notifications about important activity on your files \u2022 Use your device\u2019s camera to scan paper documents  Google Workspace subscribers have access to additional Drive functionality, including: \u2022 Easily managing users and file sharing to help meet data compliance needs \u2022 Sharing files and folders directly with groups or teams within your organization \u2022 Creating a shared drive to store all of your team\u2019s content  Learn more about Google Workspace Drive: https://workspace.google.com/products/drive/  Learn more about Google Apps update policy: https://support.google.com/a/answer/6288871  Google accounts get 15GB of storage, shared across Google Drive, Gmail, and Google Photos. For additional storage, you can upgrade to Google Workspace or Google One as an in-app purchase. Subscriptions start at $1.99/month for 100 GB in the US, and can vary by region.  Google Privacy Policy: https://www.google.com/intl/en_US/policies/privacy Google Drive Terms of Service: https://www.google.com/drive/terms-of-service  Follow us for more: Twitter: https://twitter.com/googleworkspace Linkedin: https://www.linkedin.com/showcase/googleworkspace Facebook: https://www.facebook.com/googleworkspace/" #@param {type: "string"}

app_topic_ids = []

# Avoid some special characters, such as emoji, crash the model.
description = description.encode("unicode_escape").decode("utf-8")

options = text.BertNLClassifierOptions(
    base_options=core.BaseOptions(
        file_name="model.tflite"))

tflite_topics = text.BertNLClassifier.create_from_options(options)

# The current model have the maximum length of string.
topics = tflite_topics.classify(clean_description(description)[:1200])

def CategorySort(elem):
  return elem.score
  
cats = sorted(topics.classifications[0].categories, key=CategorySort)[-_NUMBER_OF_TOP_TOPICS:][::-1]
app_topic_ids = [int(c.category_name) for c in cats if c.score >= _THRESHOLD]

if not app_topic_ids:
  print("No category was found for this input.")
else:
  print("The Android TopicsAPI will show these categories:")
  for _, app_topic_id in enumerate(app_topic_ids):
    print("Category: {} \t".format( cat_map[app_topic_id]))

The Android TopicsAPI will show these categories:
Category: /Computers & Electronics/Software/Software Utilities 	
